# Plotly Interactive Plots

Some interactive plots to visualize the dataset with more details.

In [1]:
# imports

%load_ext autoreload
%autoreload 2

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# plotly imports
from plotly.subplots import make_subplots
import plotly.graph_objs as go

from tqdm import tqdm
tqdm.pandas()

In [42]:
# Test API function before running larger simulations
from examples.sim_trace import generate_trace_api

input_data = {
    "arrival_rate": 0.9, 
    "warm_service_rate": 1/2.016, 
    "cold_service_rate": 1/2.163,
    "expiration_threshold": 600, 
    "max_time": 1e5,
}

generate_trace_api(input_data)

{'reqs_cold': 123,
 'reqs_total': 89763,
 'reqs_warm': 89640,
 'prob_cold': 0.0013702750576518165,
 'reqs_reject': 0,
 'prob_reject': 0.0,
 'lifespan_avg': 6525.9729126852935,
 'inst_count_avg': 7.763424564582287,
 'inst_running_count_avg': 1.8136152778070325,
 'inst_idle_count_avg': 5.949809286775254,
 'arrival_rate': 0.9,
 'warm_service_rate': 0.49603174603174605,
 'cold_service_rate': 0.46232085067036527,
 'expiration_threshold': 600,
 'max_time': 100000.0}

In [43]:
from pacssim.ServerlessSimulator import ServerlessSimulator

getArrayIdxArr = lambda x, idx: np.array(x)[idx]
def sample_sim_history_idxs(sim, num_of_points=200):
    hist_idx = 0
    # go in 10 second steps
    hist_step = sim.max_time / num_of_points
    idxs = [0]
    last_hist_time = 0
    while hist_idx < (len(sim.hist_times) - 1):
        hist_idx += 1
        if sim.hist_times[hist_idx] - last_hist_time < hist_step:
            continue

        last_hist_time = sim.hist_times[hist_idx]
        idxs.append(hist_idx)
    return idxs


# generate the trace
sim = ServerlessSimulator(**input_data)
sim.generate_trace(debug_print=False, progress=False)
# analyze the trace for plot
idxs = sample_sim_history_idxs(sim)
sampled_hist_times = getArrayIdxArr(sim.hist_times, idxs)
sampled_hist_inst_counts = getArrayIdxArr(sim.hist_server_count, idxs)
# calculate sampled instance count average so far
sampled_hist_inst_avgs = np.cumsum(sampled_hist_inst_counts) / np.array(list(range(len(sampled_hist_inst_counts))))

In [44]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = [i/60 for i in sampled_hist_times],
        y = sampled_hist_inst_counts,
        mode = 'markers+lines',
        name = "Current Value",
    ),
)
fig.add_trace(
    go.Scatter(
        x = [i/60 for i in sampled_hist_times],
        y = sampled_hist_inst_avgs,
        mode = 'markers+lines',
        name = "Average Estimate",
    ),
)
fig.update_layout(title="Instance Counts Over Time", xaxis=dict(title="Time (minutes)"), yaxis=dict(title="Instance Count"))

In [47]:
inst_count_avg_estimates = []
for _ in tqdm(range(10)):
    # generate the trace
    sim = ServerlessSimulator(**input_data)
    sim.generate_trace(debug_print=False, progress=False)
    # analyze the trace for plot
    idxs = sample_sim_history_idxs(sim, num_of_points=20)
    sampled_hist_times = getArrayIdxArr(sim.hist_times, idxs)
    sampled_hist_inst_counts = getArrayIdxArr(sim.hist_server_count, idxs)
    # calculate sampled instance count average so far
    sampled_hist_inst_avgs = np.cumsum(sampled_hist_inst_counts) / np.array(list(range(len(sampled_hist_inst_counts))))
    inst_count_avg_estimates.append({
        'x': [i/60 for i in sampled_hist_times],
        'y': list(sampled_hist_inst_avgs)
    })

100%|██████████| 10/10 [00:40<00:00,  4.07s/it]


In [49]:
fig = go.Figure()
for inst_count_avg_estimate in inst_count_avg_estimates:
    fig.add_trace(
        go.Scatter(
            x = inst_count_avg_estimate['x'],
            y = inst_count_avg_estimate['y'],
            mode = 'markers+lines',
            # name = "Average Estimate",
        ),
    )
fig.update_layout(title="Instance Counts Over Time", xaxis=dict(title="Time (minutes)"), yaxis=dict(title="Instance Count Average Estimate"))

fig.update_yaxes(range=[
    math.floor(min([min(i['y'][1:]) for i in inst_count_avg_estimates]) * 0.8), 
    math.ceil(max([max(i['y'][1:]) for i in inst_count_avg_estimates]) * 1.1)
    ])

## Make The Interactive Overall Plots

In [5]:
# Plot characteristics for different arrival rates and expiration thresholds
num_arrival_rates = 10
exp_thresholds = [10, 60, 600, 1200, 1800]
exp_threshold_labels = ["10 sec", "1 min", "10 min", "20 min", "30 min"]
input_data['arrival_rate'] = list(np.repeat(np.logspace(-3,1,num_arrival_rates), len(exp_thresholds)))
input_data['expiration_threshold'] = exp_thresholds * num_arrival_rates
input_data['expiration_threshold_labels'] = exp_threshold_labels * num_arrival_rates

df = pd.DataFrame(data=input_data).reset_index(drop=True)

def generate_trace_apply(x):
    return pd.Series(generate_trace_api(x.to_dict()))

df = df.progress_apply(generate_trace_apply, axis=1)
df.tail()

100%|██████████| 50/50 [00:57<00:00,  1.16s/it]


,reqs_cold,reqs_total,reqs_warm,prob_cold,reqs_reject,prob_reject,lifespan_avg,inst_count_avg,inst_running_count_avg,inst_idle_count_avg,arrival_rate,warm_service_rate,cold_service_rate,expiration_threshold,max_time,expiration_threshold_labels
45,2480,100447,97967,0.024690,0,0.0,116.248397,28.710057,20.129526,8.580531,10.0,0.496032,0.462321,10,10000,10 sec
46,387,99841,99454,0.003876,0,0.0,907.194491,33.122216,20.193303,12.928913,10.0,0.496032,0.462321,60,10000,1 min
47,70,101188,101118,0.000692,0,0.0,6209.052561,37.711720,20.269491,17.442229,10.0,0.496032,0.462321,600,10000,10 min
48,51,100855,100804,0.000506,0,0.0,6140.364174,38.837267,20.386441,18.450826,10.0,0.496032,0.462321,1200,10000,20 min
49,46,100213,100167,0.000459,0,0.0,4658.247438,39.034405,20.213882,18.820524,10.0,0.496032,0.462321,1800,10000,30 min


In [6]:
# calculate utilizations
df['utilization_percent'] = df['inst_running_count_avg'] / df['inst_count_avg'] * 100
df['prob_cold_percent'] = df['prob_cold'] * 100

## Interactive Plots

We will be making interactive plots here using the data generated by our simulator.

In [7]:
# simple interactive plot

# create a dataframe for each expiration threshold
dfs = [df.loc[df['expiration_threshold_labels'] == name, :] for name in exp_threshold_labels]

# create traces
def df_add_trace(col_name, col_label, fig, axis_titles):
    for idx in range(len(exp_threshold_labels)):
        fig.add_trace(
            go.Scatter(
                x = dfs[idx].loc[:,'arrival_rate'],
                y = dfs[idx].loc[:,col_name],
                mode = 'markers+lines',
                name = exp_threshold_labels[idx],
            ),
        )

        axis_postfix = ""
        axis_titles["xaxis" + axis_postfix] = dict(title="Arrival Rate (reqs/sec)")
        axis_titles["yaxis" + axis_postfix] = dict(title=col_label)

    return axis_titles


def plot_fig_for_col(col_name, col_label):
    fig = go.Figure()
    axis_titles = {}
    axis_titles = df_add_trace(col_name, col_label, fig, axis_titles)
    fig.update_layout(
        xaxis_type="log",
        **axis_titles
        )
    fig.show()

In [8]:
plot_fig_for_col('prob_cold_percent', 'Prob of Cold Start (%)')

In [9]:
plot_fig_for_col('utilization_percent', 'Utilization (%)')

In [10]:
plot_fig_for_col('inst_idle_count_avg', 'Average Idle Instances')